In [53]:
import tensorflow as tf;
from transformers import TFAutoModel;

In [54]:
bert=TFAutoModel.from_pretrained('bert-base-cased')

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [55]:
bert.summary()

Model: "tf_bert_model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  108310272 
Total params: 108,310,272
Trainable params: 108,310,272
Non-trainable params: 0
_________________________________________________________________


In [56]:
input_ids=tf.keras.layers.Input(shape=(512,),name="input_ids",dtype="int32")
mask=tf.keras.layers.Input(shape=(512,),name="attention_mask",dtype="int32")
#transformers
embeddings=bert.bert(input_ids,attention_mask=mask)[1]
x=tf.keras.layers.Dense(1024,activation="relu")(embeddings)
outputs=tf.keras.layers.Dense(5,activation="softmax",name="outputs")(x)
model=tf.keras.Model(inputs=[input_ids,mask],outputs=outputs)

In [57]:
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 512)]        0                                            
__________________________________________________________________________________________________
attention_mask (InputLayer)     [(None, 512)]        0                                            
__________________________________________________________________________________________________
bert (TFBertMainLayer)          TFBaseModelOutputWit 108310272   input_ids[0][0]                  
                                                                 attention_mask[0][0]             
__________________________________________________________________________________________________
dense_4 (Dense)                 (None, 1024)         787456      bert[0][1]                 

In [58]:
model.layers[2].trainable=False

In [59]:
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 512)]        0                                            
__________________________________________________________________________________________________
attention_mask (InputLayer)     [(None, 512)]        0                                            
__________________________________________________________________________________________________
bert (TFBertMainLayer)          TFBaseModelOutputWit 108310272   input_ids[0][0]                  
                                                                 attention_mask[0][0]             
__________________________________________________________________________________________________
dense_4 (Dense)                 (None, 1024)         787456      bert[0][1]                 

In [60]:
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5,decay=1e-6),
              metrics=["accuracy"])

In [61]:
element_spec=({'input_ids': tf.TensorSpec(shape=(16, 512), dtype=tf.int32, name=None),
  'attention_mask': tf.TensorSpec(shape=(16, 512), dtype=tf.int32, name=None)},
 tf.TensorSpec(shape=(16, 5), dtype=tf.float64, name=None))

In [62]:
train_data=tf.data.experimental.load('train',element_spec=element_spec)
val_data=tf.data.experimental.load('val',element_spec=element_spec)

In [63]:
history=model.fit(train_data,epochs=3,validation_data=val_data)

Epoch 1/3
426/426 [==============================] - 225s 492ms/step - loss: 1.5616 - accuracy: 0.2971 - val_loss: 1.5042 - val_accuracy: 0.3645
Epoch 2/3
426/426 [==============================] - 202s 475ms/step - loss: 1.5182 - accuracy: 0.3273 - val_loss: 1.4744 - val_accuracy: 0.3820
Epoch 3/3
426/426 [==============================] - 196s 461ms/step - loss: 1.4935 - accuracy: 0.3342 - val_loss: 1.4503 - val_accuracy: 0.3861


In [64]:
model.save("sentiment_model")

INFO:tensorflow:Assets written to: sentiment_model\assets


INFO:tensorflow:Assets written to: sentiment_model\assets
